In [99]:
import pandas as pd
import numpy as np
import nltk
import os
import nltk.corpus

#DOWNLOAD once
#nltk.download('punkt')
#nltk.download('wordnet')
#nltk.download('stopwords')
#nltk.download('words')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('maxent_ne_chunker')

In [100]:
#RETRIEVE text
#library https://newspaper.readthedocs.io/en/latest/user_guide/quickstart.html
from newspaper import Article
url = 'https://www.nrc.no/news/2020/september/yemen-900-airstrike-and-shelling-hits-on-farms-in-three-years/'
article = Article(url)
article.download()
article.parse()
bod = article.text
bodOneString = bod.replace("\n","")

In [101]:
#CHECK for governorates with p-codes
pUrl = 'https://www.humanitarianresponse.info/sites/www.humanitarianresponse.info/files/documents/files/yemen_pcodes_2015_08_29.xlsx'
data = pd.read_excel(pUrl, sheet_name="Governorates",usecols=[0,1],skiprows=[0])
govinbodIndex = []
govcodes = data["Governorate Pcode"]
govnames = data["Governorate English"]
#print(data)
for key, value in govnames.iteritems():
    if value in bodOneString:
        govinbodIndex.append(key)
print(govinbodIndex)

[6, 13, 15]


In [102]:
#LEMMING (skipping steps: remove punctuation, stop words, uppercase)
from gensim.utils import lemmatize 

lemm = set([word.decode('utf-8').split('/')[0] for word in lemmatize(bodOneString)]) 
lemmatized = list(lemm)

print(lemmatized) 

['system', 'give', 'council', 'crisis', 'be', 'shatter', 'show', 'total', 'gun', 'hajjah', 'safe', 'functional', 'restore', 'water', 'bombing', 'also', 'currency', 'time', 'risk', 'week', 'well', 'child', 'nations', 'attack', 'gulf', 'country', 'part', 'keep', 'refugee', 'onion', 'mill', 'yemeni', 'settlement', 'feature', 'family', 'baby', 'get', 'secretary', 'senseless', 'far', 'farmer', 'side', 'refer', 'village', 'few', 'many', 'happen', 'produce', 'destroy', 'crowd', 'flee', 'marib', 'yeman', 'lifeline', 'bomb', 'power', 'dependent', 'abdullah', 'abyss', 'interview', 'state', 'government', 'military', 'underfund', 'programme', 'item', 'starvation', 'higher', 'hassan', 'irrigation', 'ceasefire', 'neighbouring', 'political', 'home', 'civilian', 'no', 'month', 'vegetable', 'gunfire', 'fighting', 'field', 'father', 'wife', 'labourer', 'scale', 'member', 'funding', 'conflict', 'involve', 'organisation', 'generation', 'case', 'used', 'jan', 'long', 'displace', 'glad', 'decimate', 'event'

In [103]:
#SYNONYMS for indication terms
from nltk.corpus import wordnet 

#destroy
destroysyn = []   
for syn in wordnet.synsets("destroy"): 
    for l in syn.lemmas(): 
        destroysyn.append(l.name()) 
print(set(destroysyn))

{'ruin', 'demolish', 'destroy', 'destruct', 'put_down'}


In [104]:
#FIND indication terms

#level 1
levelonewords = destroysyn.copy() + ['raze','burn','damage']
levelonematch = []
for term in levelonewords:
    if term in lemmatized and term not in levelonematch:
        levelonematch.append(term)   

print(levelonematch)

['destroy']


In [105]:
#FIND infrastructure terms
infr = ['building','village','house','school','hospital','market','road','bridge']
infrmatch = []

for term in infr:
    if term in lemmatized and term not in infrmatch:
        infrmatch.append(term)
        
print(infrmatch)

['village', 'market']


In [106]:
#OUTPUT
if len(govinbodIndex) != 0 and len(levelonematch) != 0 and len(infrmatch) != 0:
    print("Yemen p-codes & cities: ")
    for i in govinbodIndex:
        print(govcodes[i], end=" ")
        print(govnames[i])

    print("\nIndication terms: ",end="")
    for x in range(len(levelonematch)):
        print(levelonematch[x], end=" ")

    print("\nInfrastructure affected: ",end="")
    for x in range(len(infrmatch)):
        print(infrmatch[x], end=" ")

    print("\nInfrastructure damage intensity (0-1): ", end="")
    if len(infrmatch) != 0:
        print(1)
else:
    print("Infrastructure damage intensity (0-1): 0")

Yemen p-codes & cities: 
17 Hajjah
24 Aden
26 Marib

Indication terms: destroy 
Infrastructure affected: village market 
Infrastructure damage intensity (0-1): 1
